# 딥러닝 기반 상품 카테고리 자동 분류 서버 예

### 파일에서 학습 데이터를 읽는다.

In [40]:
from sklearn.externals import joblib
import sys

In [41]:
import json


In [42]:
x_text_list = []
y_text_list = []
enc = sys.getdefaultencoding()
with open("refined_category_dataset.dat",encoding=enc) as fin:
    for line in fin.readlines():
#         print (line)
        info = json.loads(line.strip())
        x_text_list.append((info['pid'],info['name']))
        y_text_list.append(info['cate'])
        

In [43]:
# joblib.dump(y_name_id_dict,"y_name_id_dict.dat")

### text 형식으로 되어 있는 카테고리 명을 숫자 id 형태로 변환한다.

In [44]:
y_name_id_dict = joblib.load("y_name_id_dict.dat")

In [45]:
print(y_name_id_dict)

{'취미': 5, '뷰티': 0, '출산/육아': 16, '반려동물': 4, '디지털': 10, '도서/문구': 8, '스포츠/레저': 9, '잡화': 14, '건강': 7, '생필품/주방': 13, '의류': 2, '자동차/공구': 1, '컴퓨터': 3, '가구/인테리어': 11, '가전': 12, '식품': 6, '여행/e쿠폰': 15}


In [46]:

# y_name_set = set(y_text_list)
# y_name_id_dict = dict(zip(y_name_set, range(len(y_name_set))))
# print(y_name_id_dict.items())
# y_id_name_dict = dict(zip(range(len(y_name_set)),y_name_set))
y_list = [y_name_id_dict[x] for x in y_text_list]

### train test 분리하는 방법 

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test , y_train, y_test = train_test_split(x_text_list, y_list, test_size=0.2, random_state=42)


## 딥러닝 기반 text 분류에 필요한 모듈 로드

In [48]:
import os
import sys
import keras
import tensorflow as tf
import gensim
import keras.preprocessing.text
import numpy
import gensim
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
import numpy as np

#### 모델 파일을 만약 만들었다면, 아래와 같이 로드 가능하다.

In [49]:
# word2vec = gensim.models.KeyedVectors.load_word2vec_format('/workspace/resources/11st_all_product_name.segment.0918.15w100e3min.model', binary=True)
# word2vec.init_sims(replace=True)

### text 데이터를 word-id 형태로 변환한다.

In [80]:
X_train[:100]

[('1284373589', '도그차일드 레오파드애견신발 노랑 강아지슈즈 애'),
 ('1603361243', '불스 방청윤활제 360ML 윤활방청제 방청유 윤활유 녹'),
 ('1086777272', '[쿠스쿠스파이]블루베리치즈파이 800g(100g x 8조각)'),
 ('1367029764', '파사바체 유리 계량컵/주방용품/계량컵/계량용컵/조리도구/계량용품'),
 ('1687461732',
  '제주 코코몽에코파크+메이즈랜드  제주도 관광지 2곳 패키지 입장권 할인 /기프트제주/제주 승마체험/제주 승마/제주승마할인/제주 승마장 추천/제주도 승마체험 가격말타는곳 말타기'),
 ('1639169754', 'BA537 발목 서포트 LP-954 압박밴드 - 발목보호대 무'),
 ('1470415954', '[중고]서버용 HDD SAS Seagate ST300MP0005 300GB/15K'),
 ('1255743159', '(투핫) [헬로키티]헬로키티 쿠킹컵 세트'),
 ('1611544124', '인테리어 스위치커버 그래픽스티커 파인애플 WBSS7170'),
 ('1486501072', '멋스러운 캐주얼 정장 서스펜더 멜빵 TE1005'),
 ('1549175351', '1200M [미코아이엔티] 아트조이 DIY 명화그리기 숲속의 작은집'),
 ('1401228966', '3M  PF 프라이버시 필터 (15.6W)'),
 ('1530257201', 'LD-PA5 인도 사우디아라비아 등 해외여행 아답타'),
 ('1321872342', '[착한스포츠]STAR 고무연식 야구공(12개입)'),
 ('1578581418', '다람쥐PMC-230 고양이장난감 고양이용장난감 캣장난감'),
 ('1036567080',
  'PS무료배송 [랄라룹시 윈터드레스패션세트(506539)] (인형별매) 랄라룹시 윈터드레스 패션세트 인형옷 인..'),
 ('1651852477', '보브 듀얼 커버 CC크림 30ml 21호 라이트 톤업 커버'),
 ('1629587718', '허니버터링쿠

In [50]:


sequence_tokenizer = keras.preprocessing.text.Tokenizer()
sequence_tokenizer.fit_on_texts(map(lambda i: i[1], X_train))
max_features = len(sequence_tokenizer.word_index)


def texts_to_sequences2(d_list, tokenizer, maxlen=300):
    seq = tokenizer.texts_to_sequences(d_list)
    print('mean:', numpy.mean([len(x) for x in seq]))
    print('std:', numpy.std([len(x) for x in seq]))
    print('median:', numpy.median([len(x) for x in seq]))
    print('max:', numpy.max([len(x) for x in seq]))
    seq = keras.preprocessing.sequence.pad_sequences(seq, maxlen=maxlen, padding='post', truncating='post')
    return seq

In [51]:
from numpy import array
from numpy import argmax
from keras.utils import to_categorical

In [52]:
train = texts_to_sequences2(map(lambda i : i[1],X_train),sequence_tokenizer)
test = texts_to_sequences2(map(lambda i : i[1],X_test),sequence_tokenizer)

mean: 7.78808823529
std: 3.50519855834
median: 7.0
max: 28
mean: 4.41352941176
std: 3.05366595726
median: 4.0
max: 23


#### word의 embedding 형태의 weight 를 초기화 한다. 

In [53]:
input_dim = train.shape[1]

input_tensor = keras.layers.Input(shape=(input_dim,), dtype='int32')


In [54]:
word_vec_dim = 100
not_ct = 0
weights = numpy.zeros((max_features + 1, word_vec_dim))
for word, index in sequence_tokenizer.word_index.items():
    if False:
        pass
#     if word in word2vec.vocab:
#         weights[index, :] = word2vec[word]
    else:
        not_ct+=1
        weights[index, :] = numpy.random.uniform(-0.25, 0.25, word_vec_dim)
# del word2vec
# del sequence_tokenizer
print (not_ct)

28627


####  학습할 레이러를 구성한다.

In [57]:
embedded = keras.layers.Embedding(input_dim=max_features + 1,
                                  output_dim=word_vec_dim, input_length=input_dim,
                                  weights=[weights],trainable=True)(input_tensor)
# embedded2 = keras.layers.Dropout(0.9)(embedded)
# embedded2 = embedded

In [62]:
tensors = []
for filter_length in [3, 5]:
    tensor = keras.layers.Convolution1D(nb_filter=50, filter_length=filter_length)(embedded)
    tensor = keras.layers.Activation('relu')(tensor)
    tensor = keras.layers.MaxPooling1D(pool_length=input_dim - filter_length + 1)(tensor)
    tensor = keras.layers.Flatten()(tensor)
    tensors.append(tensor)

/usr/local/var/pyenv/versions/3.4.3/envs/selenium/lib/python3.4/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=50, kernel_size=3)`
  app.launch_new_instance()
/usr/local/var/pyenv/versions/3.4.3/envs/selenium/lib/python3.4/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=298)`
/usr/local/var/pyenv/versions/3.4.3/envs/selenium/lib/python3.4/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=50, kernel_size=5)`
  app.launch_new_instance()
/usr/local/var/pyenv/versions/3.4.3/envs/selenium/lib/python3.4/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=296)`


In [63]:
# embedded = keras.layers.Dropout(0.5)(embedded)
output_tensor = keras.layers.merge(tensors, mode='concat', concat_axis=1)
# output_tensor = keras.layers.Dropout(0.5)(output_tensor) # 0.7312
output_tensor = keras.layers.Dropout(0.5)(output_tensor) 
output_tensor = keras.layers.Dense(len(set(y_list)), activation='softmax')(output_tensor)

# output = Dense(NUM_CLASSES, input_dim = hidden_dim_2, activation = "softmax")(pool_rnn) # See equations (6) and (7).

cnn = keras.models.Model(input_tensor, output_tensor)
cnn.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
print(cnn.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 300)           0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 300, 100)      2862800     input_1[0][0]                    
____________________________________________________________________________________________________
conv1d_1 (Conv1D)                (None, 298, 50)       15050       embedding_1[0][0]                
____________________________________________________________________________________________________
conv1d_2 (Conv1D)                (None, 296, 50)       25050       embedding_1[0][0]                
___________________________________________________________________________________________

/usr/local/var/pyenv/versions/3.4.3/envs/selenium/lib/python3.4/site-packages/ipykernel/__main__.py:2: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app
/usr/local/var/pyenv/versions/3.4.3/envs/selenium/lib/python3.4/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [64]:
cnn.fit(train, np.asarray(to_categorical(y_train)), batch_size=60, nb_epoch=10,
        validation_data=(test, np.asarray(to_categorical(y_test))))

/usr/local/var/pyenv/versions/3.4.3/envs/selenium/lib/python3.4/site-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 6800 samples, validate on 1700 samples
Epoch 1/10
6800/6800 [==============================] - 13s - loss: 2.8792 - acc: 0.0669 - val_loss: 2.8195 - val_acc: 0.0829
Epoch 2/10
6800/6800 [==============================] - 12s - loss: 2.8169 - acc: 0.0899 - val_loss: 2.8033 - val_acc: 0.1071
Epoch 3/10
6800/6800 [==============================] - 12s - loss: 2.7695 - acc: 0.1071 - val_loss: 2.7909 - val_acc: 0.1218
Epoch 4/10
6800/6800 [==============================] - 12s - loss: 2.7357 - acc: 0.1228 - val_loss: 2.7806 - val_acc: 0.1235
Epoch 5/10
6800/6800 [==============================] - 12s - loss: 2.7039 - acc: 0.1532 - val_loss: 2.7684 - val_acc: 0.1376
Epoch 6/10
6800/6800 [==============================] - 12s - loss: 2.6703 - acc: 0.1679 - val_loss: 2.7566 - val_acc: 0.1524
Epoch 7/10
6800/6800 [==============================] - 12s - loss: 2.6348 - acc: 0.1924 - val_loss: 2.7445 - val_acc: 0.1618
Epoch 8/10
6800/6800 [==============================] - 12s - loss: 2.

In [65]:
eval_x_text_list = []
with open("soma8_test_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        eval_x_text_list.append((info['pid'],info['name']))


In [66]:
eval_x_list = texts_to_sequences2(map(lambda i : i[1],eval_x_text_list),sequence_tokenizer)


mean: 4.48684210526
std: 2.83126839769
median: 4.0
max: 18


In [77]:
eval_x_list

array([[  267,  3750,     0, ...,     0,     0,     0],
       [   13,   598,  1053, ...,     0,     0,     0],
       [ 6209,     0,     0, ...,     0,     0,     0],
       ..., 
       [16841,   319,    82, ...,     0,     0,     0],
       [ 5289,     0,     0, ...,     0,     0,     0],
       [20624,     0,     0, ...,     0,     0,     0]], dtype=int32)

In [78]:
# pred_list = clf.predict(vectorizer.transform(map(lambda i : i[1],eval_x_text_list)))
pred = cnn.predict(eval_x_list)
pred_list = [argmax(y) for y in pred]

In [79]:
import requests
name='정윤수'
nickname='시베리아_대하~'
mode='test'
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list)),
         'name':name,'nickname':nickname,'mode':mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)
print (d.json())

{'msg': 'If you pull docker image before 2017-09-27 21:30,  pull your docker image again.', 'precision': 0.20123839009287925}


### CNN 으로 추출한 이미지 데이터 사용하기 
 * 이 부분은 각자 한번 해보도록 해요

In [71]:
from PIL import Image

In [73]:
img = Image.open('/Users/yunsu/mysite-projects/soma/imgs/1002465039.jpg')

In [76]:
img.size

(530, 530)